# Cheater (Beginner)
You will notice that you have received a python file. This code was run by the challenge creator with the actual flag in a file called `secret.py`.  
Since you don't have `secret.py`, you are tasked to decrypt the file by deriving the keys using the vulnerabilities in the implementation of the cryptosystem.    

First, let us take a look at the python file.

In [ ]:
from secret import flag
from Crypto.Util.number import getPrime, bytes_to_long, long_to_bytes

m = bytes_to_long(flag)
e = 65537

n1 = 0
p1 = getPrime(256)
for i in range(1, p1*2, 2):
    n1 += i

n2 = 0
p2 = getPrime(256)
for i in range(1, p2*2, 2):
    n2 += i

n = n1 * n2
print("n =", str(n))

c = pow(m, e, n)

print("c =", str(long_to_bytes(p1 * c)))


# n = 46884503505861237425470108781013151709608385156407042535162658655219198293531603954762072725496274761369924515589793971505529848123272005525045726561087298432276907310910641955233378153675627867971707733693334181779557896862253210884460657132373758389569573567937002331226015986481271527653964521274862740761
# c = b'\x05dT!$O\xa9\xa7r\xce}l!\xf5:L\xcd\x9eq\xc9\x12\x98\x05\xc6+o\xd5\x06\x8b\xebp\x82\x1fu\xf4\x85O\xf5\xaa\x1c\xed\xc7\x86\xe8\x9dz\xa2=\x90\xe0f\xd7\xbb\x81\x1e\xf9\xab\xe71)\x1b\x994\xdd\xa7F\xc16\xff\x87\xbb/;\xea.U\xd7\x9be\xb7^\xef\x9f\xb1\x8e\xd6:\xb1\xafP\xeb5\xf4\xe5\x94\xd0\x8a\xd9[O\x1d=r\xf9\xaf\xdf-\xc3k\xb9\x95\x90\nZ:\x07"\xa5\' \x80\xc0\'R\x9a\xd5`7/\x89\xbf\x7f$u\x08\xecZw\xa6*\xb2\xe4Frr\xbd\xf5G\xf9\xcf\x90\x9a\xd6\xb5S\xdc]\xebB\xf7'

You will notice the two lines `# n = 468...` and `# c = b'\x05...`. These are the two outputs that were produced when the challenge creator ran the program.

Let us first understand this cryptosystem. This is an implementation of the [RSA](https://en.wikipedia.org/wiki/RSA_cryptosystem) cryptosystem.  
This cryptosystem is an asymmetric cryptosystem, meaning it uses the public and private keys respectively (different keys) for encryption and decryption.  
In this cryptosystem, encryption is performed by the following operation:
$$c := m^e \mod n$$
where  
- $c$ is the ciphertext
- $m$ is the plaintext being encrypted, converted into an integer using `bytes_to_long`
- $(e,n)$ are the public keys

As for the private key, it is often denoted by $d$, where
$$d = e^{-1} \mod \phi(n)$$
$\phi$ denotes Euler's totient function, which, for a semiprime $n$ i.e. $n$ is a factor of two primes, which is the standard type of n value used in RSA, is equals to $(p-1)(q-1)$.  
You can decrypt with the following operation:
$$m := c^d \mod n$$
With that being said, most CTF challenges on RSA task you to obtain the value of p and q through vulnerabilities.

Let us take a look at the code now.

In [ ]:
m = bytes_to_long(flag)
e = 65537

These 2 lines are part of the standard implementations of the RSA algorithm. $e$ is commonly set to $65537$ to allow the modular exponentiation to take less time to run, the details of which I will not be elaborating on here.

In [ ]:
n1 = 0
p1 = getPrime(256)
for i in range(1, p1*2, 2):
    n1 += i

n2 = 0
p2 = getPrime(256)
for i in range(1, p2*2, 2):
    n2 += i

n = n1 * n2
print("n =", str(n))

From this block of code, we can notice that `n1` and `n2` are the products of the first `p1` and `p2` odd numbers respectively.

In [ ]:
c = pow(m, e, n)

print("c =", str(long_to_bytes(p1 * c)))

Lastly, this is just standard RSA decryption, as mentioned above, with the additional step of converting c from an integer to bytes, which can be easily reversed with `bytes_to_long`.

Time to start on the actual challenge solving.  
Firstly, we can notice that the value of $c$ printed is a multiple of p1, as indicated by `print("c =", str(long_to_bytes(p1 * c)))`.  
This means we can leak p1 using `gcd`, or greatest common divisor, as long as we have another multiple of p1. Normally, with textbook RSA, this would be trivial, because $n=p1*p2$, both of which are primes.  
Except that we do actually have a multiple of $p1$! A simple google search will tell you that the sum of the first $n$ odd numbers is $n^2$. This means that $n = p1^2*p2^2$.    

Now, your turn! You have all the necessary information to recover $p1$, so go ahead and do so!  
Note: if you would like to install the `Crypto.Util.number` library, you can run `!pip3 install pycryptodome` in a new cell.

In [ ]:
# feel free to code here

Now that we have $p1$, you can recover $p2$, given that $n=p1^2*p2^2$.

In [ ]:
# feel free to code here

Now that we have $p1$ and $p2$, we are free to decrypt, all we need to do is derive $d$.  
Earlier I mentioned that $\phi(n)=(p-1)(q-1)$ for a semiprime $n$. Except that in our case, $n$ is not semiprime.  However, retrieving $\phi(n)$ is still doable. Think of it this way. If $p*q=n$, $\phi(n)=\phi(p)*\phi(q)$. This of course does mean that $\phi(p) = p-1$ if p is a prime.  
The last bit of information that you need is that $\phi(p^k) = (p-1)(p^{k-1})$.  
Recall that $d = e^{-1} \mod \phi(n)$. You now have enough information to derive $d$.

In [ ]:
# feel free to code here

For the last part: recall that decrypt is performed by $m := c^d \mod n$. Remember to convert m to plaintext by using `long_to_bytes` and you have your flag!

In [ ]:
# feel free to code here